In [1]:
import numpy as np
from skimage.util.shape import view_as_windows
from numpy.lib.stride_tricks import as_strided

##### Input Layer

In [2]:
input_data_list = [[-1, 1, 1, 1,-1],
                   [-1, 1,-1, 1,-1],
                   [-1, 1, 1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1, 1,-1,-1],
                   [-1, 1,-1,-1,-1]]

# Convert list to input data matrix
input_data_matrix = np.asarray(a=input_data_list, dtype=np.float32)

print(input_data_matrix.shape)

(7, 5)


##### Convolution Layer(convolution operation)

In [3]:
box_filter_list = [[1, 1, 1],
                   [1,-1, 1],
                   [1, 1, 1]]

vertical_line_filter_list = [[-1, 1,-1],
                             [-1, 1,-1],
                             [-1, 1,-1]]

diagonal_line_filter_list = [[-1,-1, 1],
                             [-1, 1,-1],
                             [ 1,-1,-1]]

In [4]:
# Convert all filter lists into matrices
box_filter_matrix = np.asarray(box_filter_list, dtype=np.float32)
vertical_line_filter_matrix = np.asarray(vertical_line_filter_list, dtype=np.float32)
diagonal_line_filter_matrix = np.asarray(diagonal_line_filter_list, dtype=np.float32)

print(f"box_filter_matrix's shape : {box_filter_matrix.shape}")
print(f"vertical_line_filter_matrix's shape : {vertical_line_filter_matrix.shape}")
print(f"diagonal_line_filter_matrix's shape : {diagonal_line_filter_matrix.shape}")

box_filter_matrix's shape : (3, 3)
vertical_line_filter_matrix's shape : (3, 3)
diagonal_line_filter_matrix's shape : (3, 3)


In [5]:
# Extract each window from input matrix by stride operation
def strided4D_v2(input_matrix, kernel_matrix, stride):
    return view_as_windows(input_matrix, kernel_matrix.shape, step=stride)

In [6]:
# Calculate shape of the feature map (output matrix from convolution layer)   # (5, 3, 3, 3) : feature map's shape is (5, 3) ==> (5, 3, 3, 3)
featureMap_row = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[0]  # 5
featureMap_col = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[1]  # 3

print(f"featureMap_row : {featureMap_row},   featureMap_col : {featureMap_col}")

featureMap_row : 5,   featureMap_col : 3


In [7]:
# Calculate featuremap matrix for box filter
def conv2d(input_matrix, kernel_matrix):
    # Create blank featuremap matrix for stride 1
    featureMap_output = np.zeros((featureMap_row, featureMap_col))
    
    for row in range(featureMap_row):
        for col in range(featureMap_col):
            window = strided4D_v2(input_matrix, kernel_matrix, 1)[row][col]
            featureMap_output[row, col] = np.sum(np.multiply(kernel_matrix, window))
            
            # To format floats in numpy array
            np.set_printoptions(precision=2)
            
            # Take average with divided by 9(total number of elements in filter matrix)
            total_num_of_elements_in_filter_matrix = kernel_matrix.shape[0] * kernel_matrix.shape[1]
            
    return featureMap_output / total_num_of_elements_in_filter_matrix

##### Box Filter Operation

In [8]:
featureMap_box = conv2d(input_data_matrix, box_filter_matrix)
print(featureMap_box)

[[-0.11  1.   -0.11]
 [-0.56  0.11 -0.33]
 [-0.33  0.33 -0.33]
 [-0.56 -0.11 -0.56]
 [-0.33 -0.56 -0.33]]


##### Vertical Filter Operation

In [9]:
featureMap_vertical = conv2d(input_data_matrix, vertical_line_filter_matrix)
print(featureMap_vertical)

[[ 0.56 -0.56  0.56]
 [ 0.56 -0.56  0.78]
 [ 0.33 -0.33  0.78]
 [ 0.11  0.11  0.56]
 [ 0.33  0.11  0.33]]


##### Diagonal Filter Operation

In [10]:
featureMap_diagonal = conv2d(input_data_matrix, diagonal_line_filter_matrix)
print(featureMap_diagonal)

[[ 0.11 -0.56  0.11]
 [ 0.11 -0.11 -0.11]
 [ 0.33 -0.33 -0.11]
 [ 0.11  0.11  0.56]
 [-0.11  1.   -0.11]]


##### Box ReLU Operation

In [49]:
featureMap_box_relu = np.maximum(featureMap_box, 0)
featureMap_vertical_relu = np.maximum(featureMap_vertical, 0)
featureMap_diagonal_relu = np.maximum(featureMap_diagonal, 0)

# np.set_printoptions(suppress=True, linewidth=300)
np.set_printoptions(formatter={'float_kind':'{:.2f}'.format}, linewidth=300)
all_kinds_relu = np.concatenate((featureMap_box_relu, np.empty((5,1)), featureMap_vertical_relu, np.empty((5,1)), featureMap_diagonal_relu), axis=1)
print(all_kinds_relu)


[[0.00 1.00 0.00 0.00 0.56 0.00 0.56 0.00 0.11 0.00 0.11]
 [0.00 0.11 0.00 0.00 0.56 0.00 0.78 0.00 0.11 0.00 0.00]
 [0.00 0.33 0.00 0.00 0.33 0.00 0.78 0.00 0.33 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.11 0.11 0.56 0.00 0.11 0.11 0.56]
 [0.00 0.00 0.00 0.00 0.33 0.11 0.33 0.00 0.00 1.00 0.00]]


##### Pooling Layer

In [57]:
def pool2d(input_matrix, kernel_size, stride, padding, pool_mode='max'):
    
    # padding
    input_matrix = np.pad(input_matrix, padding, mode='constant')
    
    # window view of input_matrix
    output_shape = ((input_matrix.shape[0]-kernel_size)//stride + 1, (input_matrix.shape[1]-kernel_size)//stride + 1)
    kernel_size = (kernel_size, kernel_size)
    # as_strided : 주어진 모양 및 보폭으로 배열을 만듦
    input_matrix_w = as_strided(input_matrix, shape=output_shape + kernel_size,
                                strides = (stride * input_matrix.strides[0], stride * input_matrix.strides[1]) + input_matrix.strides)
    input_matrix_w = input_matrix_w.reshape(-1, *kernel_size)
    
    # Return the result of pooling
    # Max Pooling
    if pool_mode == "max":
        return input_matrix_w.max(axis=(1,2)).reshape(output_shape)
    # Average Pooling
    elif pool_mode == "avg":
        return input_matrix_w.mean(axis=(1,2)).reshape(output_shape)   

In [60]:
# Max Pooling with 2*2 filter & shape=1
featureMap_box_relu_maxpool = pool2d(featureMap_box_relu, kernel_size=2, stride=1, padding=0, pool_mode="max")
featuremap_vertical_relu_maxpool = pool2d(featureMap_vertical_relu, kernel_size=2, stride=1, padding=0, pool_mode="max")
featureMap_diagonal_relu_maxpool = pool2d(featureMap_diagonal_relu, kernel_size=2, stride=1, padding=0, pool_mode="max")

np.set_printoptions(formatter={'float_kind':'{:.2f}'.format}, linewidth=300)
all_kinds_relu = np.concatenate((featureMap_box_relu_maxpool, np.empty((4,1)), featuremap_vertical_relu_maxpool, np.empty((4,1)), featureMap_diagonal_relu_maxpool), axis=1)
print(all_kinds_relu)

[[1.00 1.00 0.00 0.56 0.78 0.00 0.11 0.11]
 [0.33 0.33 0.00 0.56 0.78 0.00 0.33 0.00]
 [0.33 0.33 0.00 0.33 0.78 0.00 0.33 0.56]
 [0.00 0.00 0.00 0.33 0.56 0.00 1.00 1.00]]


In [56]:
# Imagine an array of 32-bit integers (each 4 bytes)
# The strides of an array tell us how many bytes we have to skip in memory to move to the next position along a certain axis. 
# For example, we have to skip 4 bytes (1 value) to move to the next column, but 20 bytes (5 values) to get to the same position in the next row. 
# As such, the strides for the array x will be (20, 4)
x = np.array([[0, 1, 2, 3, 4],[5, 6, 7, 8, 9]], dtype=np.int32)
print(x)
print(f"x의 shape : {x.shape}")
print(f"x의 strides : {x.strides}")

[[0 1 2 3 4]
 [5 6 7 8 9]]
x의 shape : (2, 5)
x의 strides : (20, 4)
